In [9]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
from nltk.tokenize import TweetTokenizer
from nltk.tokenize import TweetTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
import seaborn as sns

In [10]:
df = pd.read_csv("cyberbullying_tweets.csv")

In [11]:
df

,tweet_text,cyberbullying_type
0,"In other words #katandandre, your food was cra...",not_cyberbullying
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying
...,...,...
47687,"Black ppl aren't expected to do anything, depe...",ethnicity
47688,Turner did not withhold his disappointment. Tu...,ethnicity
47689,I swear to God. This dumb nigger bitch. I have...,ethnicity
47690,Yea fuck you RT @therealexel: IF YOURE A NIGGE...,ethnicity


In [12]:
#Preprocessing data
df.dropna(inplace=True)
df['tweet_text'] = df['tweet_text'].str.replace(r'\s*@\w+\s*', '', regex=True)                  #Removing @words
df['tweet_text'] = df['tweet_text'].str.replace(r'https?://\S+|http?://\S+', '', regex=True)    #Removing URLs
df['tweet_text'] = df['tweet_text'].str.lower()                                                 #Lowercasing all words
tweet_tokenizer = TweetTokenizer()
df['tokenized_tweet'] = df['tweet_text'].apply(lambda x: tweet_tokenizer.tokenize(x))                      #tokenized tweets //Time Consuming
lemmatizer = WordNetLemmatizer()                                                                #Lemmatize eg. doing = do,going=go //alternate:stemming
df['tokenized_tweet'] = df['tokenized_tweet'].apply(lambda x: [lemmatizer.lemmatize(word, pos='v') for word in x]) 


LookupError: 
**********************************************************************
  Resource [93mwordnet[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('wordnet')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/wordnet[0m

  Searched in:
    - '/Users/adeel/nltk_data'
    - '/Users/adeel/opt/anaconda3/nltk_data'
    - '/Users/adeel/opt/anaconda3/share/nltk_data'
    - '/Users/adeel/opt/anaconda3/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [14]:
X_train, X_test, y_train, y_test = train_test_split(df['tweet_text'], df['cyberbullying_type'], test_size=0.2, random_state=42)

# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer(max_features=5000)  # You can adjust max_features based on your dataset size
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [15]:
svm_model = SVC(kernel='linear', C=1.0)
svm_model.fit(X_train_tfidf, y_train)

SVC(kernel='linear')

In [16]:
y_pred = svm_model.predict(X_test_tfidf)

# Evaluation
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Classification Report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.84

Classification Report:
                     precision    recall  f1-score   support

                age       0.96      0.98      0.97      1603
          ethnicity       0.98      0.98      0.98      1603
             gender       0.90      0.83      0.86      1531
  not_cyberbullying       0.64      0.52      0.57      1624
other_cyberbullying       0.60      0.77      0.68      1612
           religion       0.97      0.95      0.96      1566

           accuracy                           0.84      9539
          macro avg       0.84      0.84      0.84      9539
       weighted avg       0.84      0.84      0.84      9539

